In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import time
import copy
import sys
import torch.nn.init as init
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(torch.cuda.get_device_name(0))

In [ ]:
runs_list = next(os.walk('../Data/Images'))[1]

In [ ]:
path = "../Data/Labels.csv"
header = ['Run','AD']
Labelsdf = pd.read_csv(path, names=header, usecols=[1,2], skiprows=1, index_col=False)

In [ ]:
labels_dict = dict(zip(Labelsdf.Run, Labelsdf.AD))

In [ ]:
import re

# process then save the 3d image ndarray as binary
# Images of 96, 96, 48 from 256, 176, 256

# Crop in the x and y direction (get center/hippocampi area)
# [80:176, 40:136, ]

img_dir = "../Data/Images"
dest_dir = "../Data/Processed"

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
    print("Created ouput directory: " + dest_dir)

    depth_scale = 2

    imgs, labels = [], []

    for run in runs_list:
        run_dir = os.path.join(img_dir, run)
        run_imgs = []
        for filename in os.listdir(run_dir):
            img_slice = cv2.imread(os.path.join(run_dir, filename), cv2.IMREAD_GRAYSCALE)
            # if image is square? IF DO THIS, NEED TO ACCOUNT FOR CENTER OFF SET (176 X 240)
            if  img_slice is None or img_slice.shape[0] == img_slice.shape[1]:
                print(run)
                print('1')
                break
            img_num = int(filename[-7:-4])
            if (100 <= img_num < 196):
                img_slice = cv2.imread(os.path.join(run_dir, filename), cv2.IMREAD_GRAYSCALE)
                #img_slice = cv2.resize(img_slice, (0,0), fx=1/img_scale, fy=1/img_scale, interpolation=cv2.INTER_AREA)
                img_slice = img_slice[80:176,40:136]
                # normalize pixel intensity to range of 0 and 1
                img_slice = img_slice/256
                run_imgs.append(img_slice)

        if len(run_imgs) == 0 or len(run_imgs) != 96:
            print(run)
            print('2')
            continue

        temp_arr = np.array(run_imgs)

        final_slices = []

        for y in range(temp_arr.shape[2]):
            xz_pane = temp_arr[:, :, y]
            scaled_xz = cv2.resize(xz_pane, (0, 0), fy=1/depth_scale, fx=1, interpolation=cv2.INTER_AREA)
            final_slices.append(scaled_xz)

        if not len(final_slices[0]) == 48:
            print(run)
            print('3')
            continue

        final_slices = np.dstack(final_slices)
        final_slices = np.expand_dims(final_slices, axis=0)
        if run in labels_dict:
            # save as binary
            np.save(os.path.join(dest_dir, run), final_slices)
            imgs.append(run)
            labels.append(labels_dict[run])
        else:
            continue

    imgs = np.array(imgs)
    labels = np.array(labels)

    np.save(os.path.join(dest_dir, 'imgs'), imgs)
    np.save(os.path.join(dest_dir, 'labels'), labels)

            
else:
    imgs = np.load(os.path.join(dest_dir, 'imgs.npy'))
    labels = np.load(os.path.join(dest_dir, 'labels.npy'))

In [ ]:
# Models to choose from
model_name = "vgg3d"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 100

In [ ]:
train_imgs, train_labels = [], []
val_imgs, val_labels = [], []
test_imgs, test_labels = [], []

for idx, ad in np.ndenumerate(labels):
    if (idx[0] % 10) < 6:
        train_imgs.append(imgs[idx[0]])
        train_labels.append(ad)
    if ((idx[0] % 10) >= 6) and ((idx[0] % 10) < 8):
        val_imgs.append(imgs[idx[0]])
        val_labels.append(ad)
    if ((idx[0] % 10) >= 8) and ((idx[0] % 10) <= 9):
        test_imgs.append(imgs[idx[0]])
        test_labels.append(ad)
        
train_imgs = np.array(train_imgs)
train_labels = np.array(train_labels)
val_imgs = np.array(val_imgs)
val_labels = np.array(val_labels)
test_imgs = np.array(test_imgs)
test_labels = np.array(test_labels)
        
ad_imgs, ad_labels = [], []

for idx, ad in np.ndenumerate(train_labels):
    if ad:
        ad_imgs.append(imgs[idx[0]])
        ad_labels.append(ad)

ad_imgs = np.array(ad_imgs)
ad_labels = np.array(ad_labels)

normal_ratio = train_imgs.shape[0] // ad_imgs.shape[0]
print("Normal ratio: " + str(normal_ratio))

train_imgs = np.concatenate((train_imgs, np.repeat(ad_imgs, normal_ratio - 1, axis=0)), axis=0)
train_labels = np.concatenate((train_labels, np.repeat(ad_labels, normal_ratio -1, axis=0)), axis=0)

In [ ]:
class T1Dataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = np.load(os.path.join(dest_dir, self.data[index]) + ".npy")
        x = torch.from_numpy(x).float()
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = T1Dataset(train_imgs, train_labels)
val_dataset = T1Dataset(val_imgs, val_labels)
test_dataset = T1Dataset(test_imgs, test_labels)

image_datasets = {'train': train_dataset, 'val': val_dataset, 'test': test_dataset}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size,
                                                   pin_memory=True,
                                                   shuffle=True, num_workers=0) for x in ['train', 'val', 'test']}

In [ ]:
class VGG3D(nn.Module):

    def __init__(self):
        super(VGG3D, self).__init__()
        
        self.features3d = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #256
            
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #128
            
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #64
            
            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #32
            
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #16
        )
        
        self.avgpool = nn.AdaptiveAvgPool3d((1, 3, 3))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 3 * 3, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features3d(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            normal_corrects, ad_corrects = 0, 0
            normal_count, ad_count = 0, 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                correctness = preds == labels.data
                running_corrects += torch.sum(correctness)

                index = 0
                for condition in labels.data:
                    
                    if condition == 0:
                        if correctness[index] == 1:
                            normal_corrects += 1
                        normal_count += 1
                    if condition == 1:
                        if correctness[index] == 1:
                            ad_corrects += 1
                        ad_count += 1
                    index += 1
    
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_normal_acc = normal_corrects / normal_count
#             print(normal_corrects, normal_count)
            epoch_ad_acc = ad_corrects / ad_count
#             print(ad_corrects, ad_count)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('Normal Acc: {:.4f} AD Acc: {:.4f}'.format(epoch_normal_acc, epoch_ad_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 256

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    elif model_name == "vgg3d":
        model_ft = VGG3D()
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
print(scratch_model)
# scratch_model.apply(weight_init)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.01, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
shist = []

from matplotlib.pyplot import figure
shist = [h.cpu().numpy() for h in scratch_hist]

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 5.0))
plt.legend()
plt.show()